In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

まずはデータの読み込み（プレミアリーグのデータセット）

In [2]:
raw_data_1 = pd.read_csv('./data/PremierLeague/2019-20.csv')
raw_data_2 = pd.read_csv('./data/PremierLeague/2018-19.csv')
raw_data_3 = pd.read_csv('./data/PremierLeague/2017-18.csv')
raw_data_4 = pd.read_csv('./data/PremierLeague/2016-17.csv')
raw_data_5 = pd.read_csv('./data/PremierLeague/2015-16.csv')
raw_data_6 = pd.read_csv('./data/PremierLeague/2014-15.csv')
raw_data_7 = pd.read_csv('./data/PremierLeague/2013-14.csv')
raw_data_8 = pd.read_csv('./data/PremierLeague/2012-13.csv')
raw_data_9 = pd.read_csv('./data/PremierLeague/2011-12.csv')
raw_data_10 = pd.read_csv('./data/PremierLeague/2010-11.csv')

データの形をみる

In [3]:
print(len(raw_data_1.columns))
print(len(raw_data_2.columns))
print(len(raw_data_3.columns))
print(len(raw_data_4.columns))
print(len(raw_data_5.columns))
print(len(raw_data_6.columns))
print(len(raw_data_7.columns))
print(len(raw_data_8.columns))
print(len(raw_data_9.columns))
print(len(raw_data_10.columns))

106
62
65
65
65
68
68
74
71
71


In [4]:
print(sum(raw_data_3.columns==raw_data_4.columns))

65


In [5]:
print(raw_data_1.shape)
print(raw_data_2.shape)
print(raw_data_3.shape)
print(raw_data_4.shape)
print(raw_data_5.shape)
print(raw_data_6.shape)
print(raw_data_7.shape)
print(raw_data_8.shape)
print(raw_data_9.shape)
print(raw_data_10.shape)

(288, 106)
(380, 62)
(380, 65)
(380, 65)
(380, 65)
(381, 68)
(380, 68)
(380, 74)
(380, 71)
(380, 71)


In [6]:
raw_data_6 = raw_data_6.iloc[:-1]

In [7]:
raw_data_6.shape

(380, 68)

特徴量の抽出する。ここでは日付、ホームチーム、アウェイチーム、ホームチームのゴール数、アウェイチームのゴール数、試合結果とした。

In [8]:
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
#playing_statistics_1=raw_data_1[columns_req]
playing_statistics_2=raw_data_2[columns_req]
playing_statistics_3=raw_data_3[columns_req]
playing_statistics_4=raw_data_4[columns_req]
playing_statistics_5=raw_data_5[columns_req]
playing_statistics_6=raw_data_6[columns_req]
playing_statistics_7=raw_data_7[columns_req]
playing_statistics_8=raw_data_8[columns_req]
playing_statistics_9=raw_data_9[columns_req]
playing_statistics_10=raw_data_10[columns_req]

In [9]:
playing_statistics_2.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,10/08/2018,Man United,Leicester,2,1,H
1,11/08/2018,Bournemouth,Cardiff,2,0,H
2,11/08/2018,Fulham,Crystal Palace,0,2,A
3,11/08/2018,Huddersfield,Chelsea,0,3,A
4,11/08/2018,Newcastle,Tottenham,1,2,A


In [10]:
print(len(playing_statistics_2))

380


In [11]:
def get_goals_scored(playing_stat):
    #print("get_goals_scored")
    # チーム名で辞書型を設定する
    teams = {}
    # どんなチームがあるを列挙する
    for i in playing_stat.groupby('HomeTeam').mean().T.columns: 
        #print("check {} \n".format(i))
        teams[i] = []

    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG'] #ホームチームのゴール数
        ATGS = playing_stat.iloc[i]['FTAG'] #アウェイチームのゴール数
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].HomeTeam].append(ATGS)
    
    """
    for i in teams.keys():
        print("check",len(teams[i]))
        print(teams[i])
        if len(teams[i])==31:
            teams[i]=teams[i][:-1]
            print("check",len(teams[i]))
    print("teams",len(teams["Arsenal"]))
    #print(teams)
    """

    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0

    # ゴール数を各チームごとに記録する
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

In [12]:
print(get_goals_scored(playing_statistics_2))

1   2   3   4   5   6   7   8   9   10  ...  30  31  32  33  \
Arsenal          0   2   5   6   8   8  10  10  13  14  ...  47  49  49  51   
Bournemouth      2   2   4   6  10  12  14  15  15  15  ...  45  47  49  50   
Brighton         3   5   7   9  10  12  13  13  14  14  ...  33  33  38  38   
Burnley          1   4   4   6  10  10  11  12  12  16  ...  47  49  49  51   
Cardiff          0   0   2   5   5  10  11  13  17  19  ...  47  49  49  50   
Chelsea          3   5   7   7  11  12  13  14  16  18  ...  39  42  42  44   
Crystal Palace   0   2   2   4   4   4   4   5   7   9  ...  28  30  30  31   
Everton          2   3   4   5   6   9  12  12  14  14  ...  42  44  44  45   
Fulham           0   2   6   8   9  10  11  16  16  19  ...  49  49  51  53   
Huddersfield     0   3   3   3   3   4   4   6   6   7  ...  29  29  31  32   
Leicester        2   2   3   5   8   9  10  12  13  14  ...  38  40  40  40   
Liverpool        4   4   5   5   8   8   8   8  12  13  ...  53  55 

In [13]:
def get_gss(playing_stat):
    """ゴール数、許したゴール数を記録"""
    GC = get_goals_scored(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = [] #ホームチームゴールスコア
    ATGS = [] #アウェイチームゴールスコア
    HTGC = [] #ホームチームが許したゴール数
    ATGC = [] #アウェイチームが許したゴール数

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
    
#     print("check line 87")
#     print(playing_stat.shape,len(HTGS))
    
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat

In [14]:
# print(playing_statistics_1.head())
playing_statistics_2 = get_gss(playing_statistics_2)
playing_statistics_3 = get_gss(playing_statistics_3)
playing_statistics_4 = get_gss(playing_statistics_4)
playing_statistics_5 = get_gss(playing_statistics_5)
playing_statistics_6 = get_gss(playing_statistics_6)
playing_statistics_7 = get_gss(playing_statistics_7)
playing_statistics_8 = get_gss(playing_statistics_8)
playing_statistics_9 = get_gss(playing_statistics_9)
playing_statistics_10 = get_gss(playing_statistics_10)

In [15]:
playing_statistics_2.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC
0,10/08/2018,Man United,Leicester,2,1,H,0,0,0,0
1,11/08/2018,Bournemouth,Cardiff,2,0,H,0,0,0,0
2,11/08/2018,Fulham,Crystal Palace,0,2,A,0,0,0,0
3,11/08/2018,Huddersfield,Chelsea,0,3,A,0,0,0,0
4,11/08/2018,Newcastle,Tottenham,1,2,A,0,0,0,0


In [16]:
def get_points(result):
    """結果をポイントとして返す"""
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

In [17]:
def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,31):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points

In [18]:
def get_matchres(playing_stat):
    #print("here")
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    #勝ち負け引き分け
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
    
#     for i in teams.keys():
#         print("check",len(teams[i]))
#         print(teams[i])
#         if len(teams[i])>=31:
#             teams[i]=teams[i][:30]
#             print("check",len(teams[i]))
#     print("teams",len(teams["Arsenal"]))
#     print(teams)
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [19]:
def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = [] #ホームチームポイント
    ATP = [] #アウェイチームポイント
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

In [20]:
# playing_statistics_1 = get_agg_points(playing_statistics_1)
playing_statistics_2 = get_agg_points(playing_statistics_2)
playing_statistics_3 = get_agg_points(playing_statistics_4)
playing_statistics_4 = get_agg_points(playing_statistics_5)
playing_statistics_5 = get_agg_points(playing_statistics_6)
playing_statistics_7 = get_agg_points(playing_statistics_7)
playing_statistics_8 = get_agg_points(playing_statistics_8)
playing_statistics_9 = get_agg_points(playing_statistics_9)
playing_statistics_10 = get_agg_points(playing_statistics_10)

In [21]:
print(playing_statistics_2.head())

Date      HomeTeam        AwayTeam  FTHG  FTAG FTR  HTGS  ATGS  HTGC  \
0  10/08/2018    Man United       Leicester     2     1   H     0     0     0   
1  11/08/2018   Bournemouth         Cardiff     2     0   H     0     0     0   
2  11/08/2018        Fulham  Crystal Palace     0     2   A     0     0     0   
3  11/08/2018  Huddersfield         Chelsea     0     3   A     0     0     0   
4  11/08/2018     Newcastle       Tottenham     1     2   A     0     0     0   

   ATGC  HTP  ATP  
0     0    0    0  
1     0    0    0  
2     0    0    0  
3     0    0    0  
4     0    0    0  


In [22]:
playing_statistics_2

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP
0,10/08/2018,Man United,Leicester,2,1,H,0,0,0,0,0,0
1,11/08/2018,Bournemouth,Cardiff,2,0,H,0,0,0,0,0,0
2,11/08/2018,Fulham,Crystal Palace,0,2,A,0,0,0,0,0,0
3,11/08/2018,Huddersfield,Chelsea,0,3,A,0,0,0,0,0,0
4,11/08/2018,Newcastle,Tottenham,1,2,A,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
375,12/05/2019,Liverpool,Wolves,2,0,H,65,49,65,49,3,3
376,12/05/2019,Man United,Cardiff,0,2,A,56,56,56,56,1,0
377,12/05/2019,Southampton,Huddersfield,1,1,D,56,40,56,40,0,1
378,12/05/2019,Tottenham,Everton,2,2,D,48,51,48,51,0,3


In [23]:
def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

In [24]:
def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               
        h.append(past[num-1])                    
        
        past = form.loc[at][j]              
        a.append(past[num-1])                
        
        if ((i + 1)% 10) == 0:
            j = j + 1
    
    playing_stat['HM' + str(num)] = h[:playing_stat.shape[0]]                 
    playing_stat['AM' + str(num)] = a[:playing_stat.shape[0]]

    
    return playing_stat

In [25]:
def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics   

In [26]:
# playing_statistics_1 = add_form_df(playing_statistics_1)
playing_statistics_2 = add_form_df(playing_statistics_2)
playing_statistics_3 = add_form_df(playing_statistics_3)
playing_statistics_4 = add_form_df(playing_statistics_4)
playing_statistics_5 = add_form_df(playing_statistics_5)
playing_statistics_6 = add_form_df(playing_statistics_6)
playing_statistics_7 = add_form_df(playing_statistics_7)
playing_statistics_8 = add_form_df(playing_statistics_8)
playing_statistics_9 = add_form_df(playing_statistics_9)
playing_statistics_10 = add_form_df(playing_statistics_10)

In [27]:
playing_statistics_3

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
0,13/08/16,Burnley,Swansea,0,1,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
1,13/08/16,Crystal Palace,West Brom,0,1,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
2,13/08/16,Everton,Tottenham,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
3,13/08/16,Hull,Leicester,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
4,13/08/16,Man City,Sunderland,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,21/05/17,Liverpool,Middlesbrough,3,0,H,63,38,63,38,...,W,L,D,L,W,D,L,W,W,L
376,21/05/17,Man United,Crystal Palace,2,0,H,38,49,38,49,...,D,W,L,L,L,L,D,L,D,W
377,21/05/17,Southampton,Stoke,0,1,A,37,44,37,44,...,D,L,W,D,L,D,D,L,D,W
378,21/05/17,Swansea,West Brom,2,1,H,60,48,60,48,...,W,L,W,L,D,D,W,L,L,L


In [28]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

In [29]:
# playing_statistics_1 = playing_statistics_1[cols]
playing_statistics_2 = playing_statistics_2[cols]
playing_statistics_3 = playing_statistics_3[cols]
playing_statistics_4 = playing_statistics_4[cols]
playing_statistics_5 = playing_statistics_5[cols]
playing_statistics_6 = playing_statistics_6[cols]
playing_statistics_7 = playing_statistics_7[cols]
playing_statistics_8 = playing_statistics_8[cols]
playing_statistics_9 = playing_statistics_9[cols]
playing_statistics_10 = playing_statistics_10[cols]

In [30]:
Standings = pd.read_csv("./data/PremierLeague/eplStandings.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)

def get_last(playing_stat, Standings, year):
    """ 順位結果 """
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    return playing_stat

In [31]:
# playing_statistics_1 = get_last(playing_statistics_1, Standings, 0)
playing_statistics_2 = get_last(playing_statistics_2, Standings, "18")
playing_statistics_3 = get_last(playing_statistics_3, Standings, "17")
playing_statistics_4 = get_last(playing_statistics_4, Standings, "16")
playing_statistics_5 = get_last(playing_statistics_5, Standings, "15")
playing_statistics_6 = get_last(playing_statistics_6, Standings, "14")
playing_statistics_7 = get_last(playing_statistics_7, Standings, "13")
playing_statistics_8 = get_last(playing_statistics_8, Standings, "12")
playing_statistics_9 = get_last(playing_statistics_9, Standings, "11")
playing_statistics_10 = get_last(playing_statistics_10, Standings, "10")


In [32]:
playing_statistics_3.head(30)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5,HomeTeamLP,AwayTeamLP
0,13/08/16,Burnley,Swansea,0,1,A,0,0,0,0,...,M,M,M,M,M,M,M,M,7.0,18.0
1,13/08/16,Crystal Palace,West Brom,0,1,A,0,0,0,0,...,M,M,M,M,M,M,M,M,11.0,20.0
2,13/08/16,Everton,Tottenham,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,8.0,3.0
3,13/08/16,Hull,Leicester,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,9.0
4,13/08/16,Man City,Sunderland,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,1.0,18.0
5,13/08/16,Middlesbrough,Stoke,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,19.0
6,13/08/16,Southampton,Watford,1,1,D,0,0,0,0,...,M,M,M,M,M,M,M,M,17.0,14.0
7,14/08/16,Arsenal,Liverpool,3,4,A,0,0,0,0,...,M,M,M,M,M,M,M,M,6.0,4.0
8,14/08/16,Bournemouth,Man United,1,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,12.0,2.0
9,15/08/16,Chelsea,West Ham,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,5.0,13.0


In [33]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

In [34]:
# playing_statistics_1 = get_mw(playing_statistics_1)
playing_statistics_2 = get_mw(playing_statistics_2)
playing_statistics_3 = get_mw(playing_statistics_3)
playing_statistics_4 = get_mw(playing_statistics_4)
playing_statistics_5 = get_mw(playing_statistics_5)
playing_statistics_6 = get_mw(playing_statistics_6)
playing_statistics_7 = get_mw(playing_statistics_7)
playing_statistics_8 = get_mw(playing_statistics_8)
playing_statistics_9 = get_mw(playing_statistics_9)
playing_statistics_10 = get_mw(playing_statistics_10)


In [35]:
playing_stat = pd.concat([playing_statistics_10,
                         playing_statistics_9,
                         playing_statistics_8,
                         playing_statistics_7,
                         playing_statistics_6,
                         playing_statistics_5,
                         playing_statistics_4,
                         playing_statistics_3,
                         playing_statistics_2], ignore_index=True)

In [36]:
playing_stat.head(40)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM4,HM5,AM1,AM2,AM3,AM4,AM5,HomeTeamLP,AwayTeamLP,MW
0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,9.0,20.0,1
1,14/08/10,Blackburn,Everton,1.0,0.0,H,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,15.0,7.0,1
2,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,14.0,8.0,1
3,14/08/10,Chelsea,West Brom,6.0,0.0,H,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,2.0,11.0,1
4,14/08/10,Sunderland,Birmingham,2.0,2.0,D,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,10.0,18.0,1
5,14/08/10,Tottenham,Man City,0.0,0.0,D,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,5.0,3.0,1
6,14/08/10,Wigan,Blackpool,0.0,4.0,A,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,16.0,19.0,1
7,14/08/10,Wolves,Stoke,2.0,1.0,H,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,17.0,13.0,1
8,15/08/10,Liverpool,Arsenal,1.0,1.0,D,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,6.0,4.0,1
9,16/08/10,Man United,Newcastle,3.0,0.0,H,0.0,0.0,0.0,0.0,...,M,M,M,M,M,M,M,1.0,12.0,1


In [37]:
playing_stat.to_csv("./data/PremierLeague/v1_dataset.csv",index=None)

In [38]:
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

playing_stat['HTFormPtsStr'] = playing_stat['HM1'] + playing_stat['HM2'] + playing_stat['HM3'] + playing_stat['HM4'] + playing_stat['HM5']
playing_stat['ATFormPtsStr'] = playing_stat['AM1'] + playing_stat['AM2'] + playing_stat['AM3'] + playing_stat['AM4'] + playing_stat['AM5']

playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)

In [39]:
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0

In [40]:
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    

In [41]:
playing_stat['HTWinStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ws)
playing_stat['HTWinStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ws)
playing_stat['HTLossStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ls)
playing_stat['HTLossStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ls)

playing_stat['ATWinStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ws)
playing_stat['ATWinStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ws)
playing_stat['ATLossStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ls)
playing_stat['ATLossStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ls)

playing_stat.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [42]:
playing_stat['HTGD'] = playing_stat['HTGS'] - playing_stat['HTGC']
playing_stat['ATGD'] = playing_stat['ATGS'] - playing_stat['ATGC']

playing_stat['DiffPts'] = playing_stat['HTP'] - playing_stat['ATP']
playing_stat['DiffFormPts'] = playing_stat['HTFormPts'] - playing_stat['ATFormPts']

playing_stat['DiffLP'] = playing_stat['HomeTeamLP'] - playing_stat['AwayTeamLP']

In [43]:
playing_stat.head(50)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,-11.0
1,14/08/10,Blackburn,Everton,1.0,0.0,H,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,8.0
2,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,6.0
3,14/08/10,Chelsea,West Brom,6.0,0.0,H,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,-9.0
4,14/08/10,Sunderland,Birmingham,2.0,2.0,D,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,-8.0
5,14/08/10,Tottenham,Man City,0.0,0.0,D,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,2.0
6,14/08/10,Wigan,Blackpool,0.0,4.0,A,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,-3.0
7,14/08/10,Wolves,Stoke,2.0,1.0,H,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,4.0
8,15/08/10,Liverpool,Arsenal,1.0,1.0,D,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,2.0
9,16/08/10,Man United,Newcastle,3.0,0.0,H,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,0,-11.0


In [44]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
playing_stat['FTR'] = playing_stat.FTR.apply(only_hw)

In [45]:
playing_stat=playing_stat.drop(["Date","HomeTeam","AwayTeam","HomeTeamLP","AwayTeamLP","HTFormPts","ATFormPts", "FTHG","FTAG","HTFormPtsStr","ATFormPtsStr"],axis=1)
playing_stat.head()

,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,HM2,HM3,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,H,0.0,0.0,0.0,0.0,0,0,M,M,M,...,0,0,0,0,0,0.0,0.0,0,0,-11.0
1,H,0.0,0.0,0.0,0.0,0,0,M,M,M,...,0,0,0,0,0,0.0,0.0,0,0,8.0
2,NH,0.0,0.0,0.0,0.0,0,0,M,M,M,...,0,0,0,0,0,0.0,0.0,0,0,6.0
3,H,0.0,0.0,0.0,0.0,0,0,M,M,M,...,0,0,0,0,0,0.0,0.0,0,0,-9.0
4,NH,0.0,0.0,0.0,0.0,0,0,M,M,M,...,0,0,0,0,0,0.0,0.0,0,0,-8.0


In [46]:
playing_stat.keys()

Index(['FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2',
       'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5', 'MW',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [47]:
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']

In [48]:
playing_stat.MW = playing_stat.MW.astype(float)

for col in cols:
    playing_stat[col] = playing_stat[col] / playing_stat.MW

In [49]:
# playing_stat.HTGS=(playing_stat.HTGS)/
playing_stat.HTGS=playing_stat.HTGS/playing_stat.HTGS.max()

In [50]:
playing_stat.ATGS=playing_stat.ATGS/playing_stat.ATGS.max()
playing_stat.HTGC=playing_stat.HTGC/playing_stat.HTGC.max()
playing_stat.ATGC=playing_stat.ATGC/playing_stat.ATGC.max()
playing_stat

,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,HM2,HM3,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M,M,M,...,0,0,0,0,0,0.0,0.0,0.000000,0.000000,-11.0
1,H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M,M,M,...,0,0,0,0,0,0.0,0.0,0.000000,0.000000,8.0
2,NH,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M,M,M,...,0,0,0,0,0,0.0,0.0,0.000000,0.000000,6.0
3,H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M,M,M,...,0,0,0,0,0,0.0,0.0,0.000000,0.000000,-9.0
4,NH,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M,M,M,...,0,0,0,0,0,0.0,0.0,0.000000,0.000000,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,H,0.855263,0.662162,0.855263,0.662162,0.078947,0.078947,W,W,W,...,0,0,0,0,0,0.0,0.0,0.000000,0.131579,-5.0
3416,NH,0.736842,0.756757,0.736842,0.756757,0.026316,0.000000,D,D,L,...,0,0,0,0,0,0.0,0.0,0.026316,0.052632,-12.0
3417,NH,0.736842,0.540541,0.736842,0.540541,0.000000,0.026316,L,D,D,...,0,0,0,1,0,0.0,0.0,-0.026316,0.105263,-4.0
3418,NH,0.631579,0.689189,0.631579,0.689189,0.000000,0.078947,L,L,W,...,0,0,0,0,0,0.0,0.0,-0.078947,-0.105263,-4.0


In [51]:
print(playing_stat.ATGC.max())
print(playing_stat.HTGC.max())

1.0
1.0


In [52]:
playing_stat=playing_stat.drop(["MW"],axis=1)

In [53]:
playing_stat.shape

(3420, 30)

In [54]:
#テストデータ
playing_stat_test = playing_stat[2620:]

In [55]:
playing_stat_test

,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,HM2,HM3,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
2620,NH,0.657895,0.756757,0.657895,0.756757,0.000000,0.085714,L,W,L,...,0,0,0,0,0,0.0,0.0,-0.085714,-0.028571,8.0
2621,NH,0.671053,0.662162,0.671053,0.662162,0.028571,0.028571,D,W,W,...,0,0,0,0,0,0.0,0.0,0.000000,0.171429,-14.0
2622,H,0.789474,0.554054,0.789474,0.554054,0.028571,0.000000,D,W,W,...,0,0,0,0,0,0.0,0.0,0.028571,0.171429,-10.0
2623,H,0.592105,0.459459,0.592105,0.459459,0.028571,0.000000,D,W,W,...,0,0,0,0,0,0.0,0.0,0.028571,0.171429,-3.0
2624,NH,0.460526,0.500000,0.460526,0.500000,0.028571,0.085714,D,W,L,...,0,0,0,0,0,0.0,0.0,-0.057143,-0.142857,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,H,0.855263,0.662162,0.855263,0.662162,0.078947,0.078947,W,W,W,...,0,0,0,0,0,0.0,0.0,0.000000,0.131579,-5.0
3416,NH,0.736842,0.756757,0.736842,0.756757,0.026316,0.000000,D,D,L,...,0,0,0,0,0,0.0,0.0,0.026316,0.052632,-12.0
3417,NH,0.736842,0.540541,0.736842,0.540541,0.000000,0.026316,L,D,D,...,0,0,0,1,0,0.0,0.0,-0.026316,0.105263,-4.0
3418,NH,0.631579,0.689189,0.631579,0.689189,0.000000,0.078947,L,L,W,...,0,0,0,0,0,0.0,0.0,-0.078947,-0.105263,-4.0


In [56]:
playing_stat=playing_stat[:2620]

In [57]:
playing_stat.shape

(2620, 30)

In [58]:
playing_stat.to_csv("./data/PremierLeague/final_dataset.csv",index=None)
playing_stat_test.to_csv("./data/PremierLeague/test.csv",index=None)